In [1]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk

In [2]:
with open("Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

In [4]:
print(json_list[0])

{'unixReviewTime': 1355616000, 'reviewerID': 'A10000012B7CGYKOMPQ4L', 'summary': 'Wonderful!', 'reviewText': 'Spiritually and mentally inspiring! A book that allows you to question your morals and will help you discover who you really are!', 'reviewTime': '12 16, 2012', 'overall': 5.0, 'reviewerName': 'Adam', 'asin': '000100039X', 'helpful': [0, 0]}


In [5]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)
type(review_data)

list

In [6]:
import random
random_data = random.sample(review_data,len(review_data))
random_data[0]

{'overall': 4.0,
 'reviewText': 'This is a solid effort--much better than such Leonard failures as Djibouti--but far from his best (which are among the best in the genre). Others have explained the plot well.Because this is basically 3 intertwined novellas, none of the novellas has nearly the punch of a first rate novel: none of the novellas has time for impressive twists and turns and the wrongdoers are not only not memorable, but ridiculous.  The smooth coal company lawyer who murders someone in front of a witness for little reason is the worst, but not by much.  Still, Raylan is an entertaining hero and it is a fun quick read.It is similar in quality to the tv series featuring Raylan--fun, but not nearly the quality of a good movie.'}

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jangy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"])
print(reviewText[2:4])
len(reviewText)

['"Bond Girl" was a fun book to read, Alex Garrett was a great character to follow.  When the reader is allowed to figure something out before the main character does (which happens a few times here), it\'s actually fun as we read along with the ensuing crazy train wreck, since you just wanted to reach in and slap some sense into her!I definitely enjoyed the storyline, though try as I might, I couldn\'t overlook it\'s similarities to "The Devil Wears Prada".  Girl gets a job that\'s way over her head, ends up in culture shock as all of her co-workers seem insane to her, she adjusts after much craziness, likes it for a while, then realizes her life is going down the toilet as she tries to keep up.Despite those similarities, there was a lot of intriguing content to keep this book fresh.- I learned a little about the world of finance, although I can\'t say I truly understand it after reading this.- Alex didn\'t always like living in Manhattan, which breaks with the usual romanticized vers

8898041

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, max_df = 0.3, min_df = 2, lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(reviewText)
# max_df = 0.4 , min_df 0.01 , Accuracy = 49%
# max_df = 0.4 , min_df 0.0.005 , Accuracy = 53%
# max_df = 0.4 , min_df 0.0.004 , Accuracy = ?

# 40%제거 vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, max_df = 0.3, min_df = 2, lowercase=True, stop_words = 'english')
# default : (93184, 116502)
# (93184, 60901)

In [10]:
D.shape
# 행,렬 

(8898041, 698489)

In [11]:
D = D.toarray()
print(D)
#리스트로 만듬

MemoryError: 

In [ ]:
y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
for i in y_data:
    int_y_data.append((int(i),))
print(int_y_data[:10])

In [ ]:
y_data = np.array(MultiLabelBinarizer().fit_transform(int_y_data))

In [ ]:
y_data.shape

In [ ]:
print(y_data[:10])

In [ ]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [93184, 60901])

w1 = tf.Variable(tf.truncated_normal([60901, 300], stddev = 0.1))
b1 = tf.Variable(tf.truncated_normal([300]))
w2 = tf.Variable(tf.truncated_normal([300, 5], stddev = 0.1))
b2 = tf.Variable(tf.truncated_normal([5]))

output =  tf.sigmoid(tf.matmul(x, w1) + b1)
hypothesis = tf.sigmoid(tf.matmul(output,w2)+b2)

cost = -tf.reduce_sum(y_data * tf.log(tf.clip_by_value(hypothesis,1e-10,1.0)))
train_step = tf.train.AdamOptimizer(0.01).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(200):
    sess.run(train_step, feed_dict={x:D})
    if i % 100 == 0:
        print(i, sess.run(cost, feed_dict={x:D}))
        
# 뉴럴넷
# 0.1 , 0.01 49%
# 0.1 , 0.005 53%
# 300 : 2034, 53%
# 300 : 60901 ??

In [ ]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y_data,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: D}))
# 정확도 측정

In [ ]:
print(sess.run(hypothesis, feed_dict={x: D}))
# 결과값

In [ ]:
print(sess.run(tf.argmax(hypothesis, 1), feed_dict={x: D}))
# 결과값 중에 높은 인덱스 가져오기

### 